In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import wittgenstein as lw
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
# --- Configuration ---
DATASETS_PATH = "../../datasets/SLC/"
DATASETS = [
    "mushrooms", "tictactoe", "hepatitis", "ljubljana", "cargood", 
    "chess", "zoo3", "flare", "yeast3", "abalone19", "segment0", "pageblocks"
]

In [3]:
# --- Helper: Calculate Specificity ---
def calculate_specificity(y_true, y_pred):
    """
    Calculates Specificity (True Negative Rate). 
    For multiclass, calculates the macro-average specificity.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    # Binary case
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        return tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Multiclass case (Macro Average)
    specificities = []
    for i in range(len(cm)):
        tn = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
        fp = np.sum(cm[:, i]) - cm[i, i]
        
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificities.append(spec)
    
    return np.mean(specificities)

# --- Helper: Data Loader ---
def load_dataset(name, path):
    """
    Attempts to load dataset from path. Assumes CSV format.
    Adjust 'sep' or file extension logic if your data varies.
    """
    file_path = os.path.join(path, f"{name}.csv") # Defaulting to .csv
    
    # Fallback to .data if .csv missing, common in SLC datasets
    if not os.path.exists(file_path):
        file_path = os.path.join(path, f"{name}.data")
        
    try:
        # Trying common separators
        df = pd.read_csv(file_path, sep=None, engine='python')
        
        # Simple preprocessing: Label Encode target (last column) and categories
        le = LabelEncoder()
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = le.fit_transform(df[col].astype(str))
                
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y
    except Exception as e:
        print(f"Error loading {name}: {e}")
        return None, None

In [4]:
models = {
    "Ripper": lw.RIPPER(), 
    "C4.5": DecisionTreeClassifier(criterion='entropy'),
    "CostSensitive C4.5": DecisionTreeClassifier(criterion='entropy', class_weight='balanced')
}

In [5]:
# Storage for results
results_data = []

print(f"{'Dataset':<15} {'Algorithm':<20} {'Acc':<8} {'F1':<8} {'Recall':<8} {'Prec':<8} {'Spec':<8}")
print("-" * 85)

for dataset_name in DATASETS:
    X, y = load_dataset(dataset_name, DATASETS_PATH)
    
    if X is None:
        continue

    for model_name, model_inst in models.items():
        
        # Metrics storage for averaging across 5 runs * 5 folds = 25 scores
        run_metrics = {'acc': [], 'f1': [], 'rec': [], 'prec': [], 'spec': []}

        # 5 Runs
        for run_idx in range(5):
            # Use different random state for each run to vary the folds
            kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42 + run_idx)
            
            for train_idx, test_idx in kf.split(X, y):
                X_train, X_test = X[train_idx], X[test_idx]
                y_train, y_test = y[train_idx], y[test_idx]

                # Clone/Reset model
                if model_name == "Ripper":
                    clf = lw.RIPPER() # Wittgenstein models need fresh init
                else:
                    # Sklearn models can be cloned or re-instantiated
                    params = model_inst.get_params()
                    clf = DecisionTreeClassifier(**params)

                try:
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    # Calculate Metrics
                    # Note: 'weighted' average handles class imbalance/multiclass gracefully
                    run_metrics['acc'].append(accuracy_score(y_test, y_pred))
                    run_metrics['f1'].append(f1_score(y_test, y_pred, average='weighted', zero_division=0))
                    run_metrics['rec'].append(recall_score(y_test, y_pred, average='weighted', zero_division=0))
                    run_metrics['prec'].append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
                    run_metrics['spec'].append(calculate_specificity(y_test, y_pred))
                
                except Exception as e:
                    # Fallback for failures (common with Ripper on some data types)
                    pass

        # Aggregate Results
        res = {
            "Dataset": dataset_name,
            "Algorithm": model_name,
            "Accuracy": np.mean(run_metrics['acc']),
            "F1-Score": np.mean(run_metrics['f1']),
            "Recall": np.mean(run_metrics['rec']),
            "Precision": np.mean(run_metrics['prec']),
            "Specificity": np.mean(run_metrics['spec'])
        }
        results_data.append(res)
        
        print(f"{res['Dataset']:<15} {res['Algorithm']:<20} "
              f"{res['Accuracy']:.4f}   {res['F1-Score']:.4f}   "
              f"{res['Recall']:.4f}   {res['Precision']:.4f}   {res['Specificity']:.4f}")

Dataset         Algorithm            Acc      F1       Recall   Prec     Spec    
-------------------------------------------------------------------------------------
mushrooms       Ripper               nan   nan   nan   nan   nan
mushrooms       C4.5                 0.4849   0.4588   0.4849   0.4414   0.8983
mushrooms       CostSensitive C4.5   0.4849   0.5108   0.4849   0.5484   0.9102
tictactoe       Ripper               0.9714   0.9711   0.9714   0.9729   0.9403
tictactoe       C4.5                 0.8848   0.8838   0.8848   0.8855   0.8085
tictactoe       CostSensitive C4.5   0.8879   0.8869   0.8879   0.8884   0.8109
hepatitis       Ripper               0.8026   0.7925   0.8026   0.8040   0.9026
hepatitis       C4.5                 0.7523   0.7527   0.7523   0.7584   0.8387
hepatitis       CostSensitive C4.5   0.7535   0.7532   0.7535   0.7642   0.8388
ljubljana       Ripper               0.7177   0.6650   0.7177   0.7031   0.9298
ljubljana       C4.5                 0.6154   0